In [34]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [35]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

### Phân lớp nhị phân

In [36]:
iris = load_iris()
X = iris.data[iris.target != 2]
y = iris.target[iris.target != 2]

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [38]:
input_size = 4  # Số lượng đặc trưng trong tập dữ liệu iris
hidden_size = 8  # Số lượng đơn vị ẩn trong lớp ẩn
output_size = 1  # Chỉ có một lớp đầu ra (binary classification)

In [39]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

In [40]:
model = Net(input_size, hidden_size, output_size)

In [41]:
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [42]:
num_epochs = 100

for epoch in range(num_epochs):
    # chuyen du lieu thanh tensor
    inputs = torch.from_numpy(X_train).float()
    targets = torch.from_numpy(y_train).float()

    # Forward
    outputs = model(inputs)
    loss = criterion(outputs, targets.view(-1, 1))

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print loss every 10 epochs
    if (epoch + 1) % 10 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch + 1, num_epochs, loss.item()))


Epoch [10/100], Loss: 0.6428
Epoch [20/100], Loss: 0.5902
Epoch [30/100], Loss: 0.5463
Epoch [40/100], Loss: 0.5070
Epoch [50/100], Loss: 0.4708
Epoch [60/100], Loss: 0.4372
Epoch [70/100], Loss: 0.4060
Epoch [80/100], Loss: 0.3769
Epoch [90/100], Loss: 0.3500
Epoch [100/100], Loss: 0.3251


In [43]:
with torch.no_grad():
    inputs = torch.from_numpy(X_test).float()
    targets = torch.from_numpy(y_test)

In [44]:
# Convert probabilities to binary predictions
predicted = model(inputs)
predicted = predicted.round()

# Calculate accuracy
accuracy = (predicted == targets.view(-1, 1)).sum().item() / len(targets)

print('Accuracy on test set: {:.2f}%'.format(accuracy * 100))

Accuracy on test set: 100.00%


In [48]:
sample = torch.tensor([[5.2000, 3.5000, 1.5000, 0.2000]])
predicted = model(sample)
predicted.round()

tensor([[0.]], grad_fn=<RoundBackward0>)

### Phân lớp nhiều nhãn

In [49]:
X, y = iris.data, iris.target

In [50]:
# Split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Convert numpy arrays to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.int64)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.int64)


In [51]:
class IrisNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(IrisNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [52]:
# Set hyperparameters
input_size = 4
hidden_size = 10
output_size = 3
learning_rate = 0.01
num_epochs = 1000

In [53]:
# Initialize neural network
model = IrisNet(input_size, hidden_size, output_size)

In [54]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [55]:
# Train model
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    # Backward pass and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print loss every 100 epochs
    if (epoch+1) % 100 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Epoch [100/1000], Loss: 0.7210
Epoch [200/1000], Loss: 0.5341
Epoch [300/1000], Loss: 0.4441
Epoch [400/1000], Loss: 0.3880
Epoch [500/1000], Loss: 0.3443
Epoch [600/1000], Loss: 0.3073
Epoch [700/1000], Loss: 0.2757
Epoch [800/1000], Loss: 0.2489
Epoch [900/1000], Loss: 0.2263
Epoch [1000/1000], Loss: 0.2074


In [56]:
# Evaluate model on test set
with torch.no_grad():
    # Forward pass
    outputs = model(X_test)
    _, predicted = torch.max(outputs.data, 1)

    # Calculate accuracy
    accuracy = (predicted == y_test).sum().item() / len(y_test)
    print('Accuracy on test set: {:.2f}%'.format(accuracy * 100))

Accuracy on test set: 100.00%


In [58]:
sample = torch.tensor([[5.2000, 3.5000, 1.5000, 0.2000]])
predicted = model(sample)
predicted

tensor([[ 6.0149,  2.1836, -7.5481]], grad_fn=<AddmmBackward0>)

In [60]:
np.argmax(predicted.detach().numpy())

0